In [1]:
import math
import re
import sys
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve,roc_auc_score

# device =torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:0')
device = torch.device('cpu')

In [2]:
ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始LSTM所有特征.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-10.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-45.csv')
ori_fea = ori_fea.drop(ori_fea.columns[0], axis=1)

label = [1] * 8000 + [0] * 8000

oy = pd.DataFrame(label, columns=['label'])

ori_fea

,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,TAT,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.380952,272.0,182,72,0.273016,0.155556,0.051118,0.041534,0.003215,0.006431,...,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872
1,0.409396,454.0,432,354,0.328859,0.177852,0.079125,0.035354,0.011824,0.016892,...,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872
2,0.343101,506.0,585,339,0.344322,0.191697,0.046512,0.022032,0.008589,0.031902,...,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872
3,0.461774,427.0,327,327,0.293578,0.299694,0.036923,0.046154,0.000000,0.000000,...,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872
4,0.565217,0.0,0,0,0.304348,0.347826,0.095238,0.047619,0.000000,0.000000,...,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.450256,1175.0,975,975,0.274872,0.217436,0.047276,0.057554,0.007209,0.011329,...,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872,1.513872
15996,0.430957,1757.0,1557,1557,0.280026,0.185613,0.057235,0.051447,0.004507,0.011590,...,2.037571,1.551190,0.842497,1.551190,1.431183,0.842497,1.431183,1.431183,2.037571,1.431183
15997,0.455010,3134.0,2934,2934,0.233810,0.190184,0.042974,0.057981,0.005119,0.012287,...,2.037571,1.431183,1.551190,1.431183,0.842497,1.551190,1.431183,0.842497,1.431183,1.431183
15998,0.448404,2894.0,2694,2694,0.257981,0.213066,0.044948,0.048663,0.002230,0.011524,...,2.037571,2.037571,2.037571,0.842497,1.551190,1.431183,1.431183,2.037571,2.037571,1.431183


In [84]:
def extract_sequences_from_fasta(file_path):
    headers = []
    sequences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_header = ''
        current_sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_header and current_sequence:
                    headers.append(current_header)
                    sequences.append(current_sequence)

                current_header = line[1:]
                current_sequence = ''
            else:
                current_sequence += line

        # 处理最后一个序列
        if current_header and current_sequence:
            headers.append(current_header)
            sequences.append(current_sequence)

    data = {'seqname': headers, 'seq': sequences}
    df = pd.DataFrame(data)
    return df

# 读取FASTA文件并提取序列
file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\cRNA\Sorghum bicolor.fasta'
datah = extract_sequences_from_fasta(file_path)
print(datah.shape[0])

file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\ncRNA\Sorghum bicolor.fasta'
datah_nc = extract_sequences_from_fasta(file_path)
print(datah_nc.shape[0])
datah = pd.concat([datah,datah_nc]).reset_index(drop=True)

pattern = r'(?:^|\s)([^ ]+)'  # 匹配第一个空格前的非空格字符序列作为标识部分
datah['seqname'] = [re.search(pattern, identifier).group(1) for identifier in datah['seqname']]

data = datah
datah

8657
8657


,seqname,seq
0,Sobic.008G040400.1,ATGGACCGCTTCCTGCTGTCCCGCCCACCCCTGCCCGTCCCGACCC...
1,Sobic.002G046200.1,ATGTGGACCCAGAGGAGCCACGTCGTCCCCCACCTGCCAGAGCCAA...
2,Sobic.005G155300.1,ATGGCGTGGCTCGCGCGCTCCATCGCCAACTCCCTCCTCGCTACCG...
3,Sobic.006G231266.1,ATGTCCGTCCTTCTCCTGCTCCTCCTCGGTTCTTGGTGGCTACAGA...
4,Sobic.010G097250.1,ACCGAGTTCCTGGGCCACTGCCCTGCTTCAACGGTGGCAGCCGCCA...
...,...,...
17309,URS0001BC7647_4558,ACUCCUUUGAUGAUUAGCUAUACAUCUGGAAUUACCGUCAGAUUUA...
17310,URS0001BC88CA_4558,AAGCCAAUGAUGAUUAGCUAUACAUCUGGAAUUACCGUCAGAUUUA...
17311,URS0001BC8A9F_4558,UCCAAAGGGAUCGCAUUGAUCUCACCUGCCGCCCGCCGCUGCGCCU...
17312,URS0001BC9BE7_4558,GUGAGCUGGGAAGAGGAAGAGAGGGCCUAAGAGCAGUGGGAAUGGG...


In [85]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\c_cds_test\Sorghum_bicolor',names=names,sep='\t')
print(rna_cds.shape[0])
rna_cds_nc = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\\nc_cds_test\Sorghum_bicolor',names=names,sep='\t')
print(rna_cds_nc.shape[0])

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)

datah_seqname = datah['seqname']
rna_cds_seqname = rna_cds['seqname']

# # extra_rows_index = datah_seqname.difference(rna_cds_seqname)
indexes_to_drop = datah_seqname[~datah_seqname.isin(rna_cds_seqname)].index

# 删除行
datah.drop(indexes_to_drop, inplace=True)

# 创建一个包含每个打印的元素的列表
label_data = [0] * 8655 + [1] * 8345

# 使用pandas库将列表转换为DataFrame
y = pd.DataFrame(label_data, columns=['label'])
datah.reset_index(drop=True, inplace=True)

datah.insert(0,'label',y)
print(datah.index)
print(y.index)

data3 = datah.copy()
datah

8655
8345
RangeIndex(start=0, stop=17000, step=1)
RangeIndex(start=0, stop=17000, step=1)


,label,seqname,seq
0,0,Sobic.008G040400.1,ATGGACCGCTTCCTGCTGTCCCGCCCACCCCTGCCCGTCCCGACCC...
1,0,Sobic.002G046200.1,ATGTGGACCCAGAGGAGCCACGTCGTCCCCCACCTGCCAGAGCCAA...
2,0,Sobic.005G155300.1,ATGGCGTGGCTCGCGCGCTCCATCGCCAACTCCCTCCTCGCTACCG...
3,0,Sobic.006G231266.1,ATGTCCGTCCTTCTCCTGCTCCTCCTCGGTTCTTGGTGGCTACAGA...
4,0,Sobic.010G097250.1,ACCGAGTTCCTGGGCCACTGCCCTGCTTCAACGGTGGCAGCCGCCA...
...,...,...,...
16995,1,URS0000D5619D_4558,GGCATGGGAACATGTAGGAAGG
16996,1,URS0000D5663D_4558,AAGTTATGAACATAAAAGTTG
16997,1,URS0000D72719_4558,GTCTTACACTTGATCTCTGAACTTCAGGTAAGTTCACATGTGGAAA...
16998,1,URS0000D76B3A_4558,TGGATGATGAAATGCACTGGGCTCCGAGCAGTGAAATTTGCTGTGT...


## 手动特征

In [86]:
from itertools import product
# compute k-mers features (k=1~4)
def Kmers_funct(seq, x):
    X = [None] * len(seq)
    for i in range(len(seq)):
        a = seq[i]
        t = 0
        l = []
        for index in range(len(a) - x + 1):
            t = a[index:index + x]
            if (len(t)) == x:
                l.append(t)
        X[i] = l
    return X


def nucleotide_type(k):
    z = []
    for i in product('ACGT', repeat=k):
        z.append(''.join(i))
    return z


def Kmers_frequency(seq, x):
    X = []
    char = nucleotide_type(x)
    for i in range(len(seq)):
        s = seq[i]
        frequence = []
        for a in char:
            number = s.count(a)
            char_frequence = number / (len(s) - x + 1)
            frequence.append(char_frequence)
        X.append(frequence)
    return X


feature_1mer = Kmers_funct(data.seq, 1)  # 1-mer
feature_2mer = Kmers_funct(data.seq, 2)  # 2-mer
feature_3mer = Kmers_funct(data.seq, 3)  # 3-mer

feature_1mer_frequency = Kmers_frequency(feature_1mer, 1)  # 1-mer
feature_2mer_frequency = Kmers_frequency(feature_2mer, 2)  # 2-mer
feature_3mer_frequency = Kmers_frequency(feature_3mer, 3)  # 3-mer

feature = pd.concat([pd.DataFrame(feature_1mer_frequency),
                     pd.DataFrame(feature_2mer_frequency),
                     pd.DataFrame(feature_3mer_frequency)], axis=1)
feature.columns = ['A','C','G','T','AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT','AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']
feature

,A,C,G,T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.105364,0.394636,0.388889,0.111111,0.011538,0.042308,0.038462,0.011538,0.025000,0.142308,...,0.025097,0.001931,0.001931,0.015444,0.017375,0.001931,0.000000,0.007722,0.000000,0.000000
1,0.152439,0.400407,0.270325,0.176829,0.026531,0.055102,0.044898,0.024490,0.071429,0.171429,...,0.014344,0.012295,0.008197,0.008197,0.014344,0.014344,0.004098,0.014344,0.006148,0.024590
2,0.236364,0.290909,0.281818,0.190909,0.050835,0.055387,0.071320,0.058422,0.068285,0.081184,...,0.015957,0.013678,0.025076,0.014438,0.017477,0.009119,0.003799,0.012158,0.008359,0.012918
3,0.187773,0.340611,0.302766,0.168850,0.021898,0.042336,0.094891,0.029197,0.075912,0.105109,...,0.013177,0.013177,0.008785,0.029283,0.014641,0.004392,0.001464,0.014641,0.007321,0.000000
4,0.262626,0.259259,0.262626,0.215488,0.047458,0.077966,0.077966,0.057627,0.101695,0.054237,...,0.010239,0.000000,0.023891,0.034130,0.027304,0.006826,0.010239,0.020478,0.013652,0.010239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,0.318182,0.090909,0.454545,0.136364,0.100000,0.050000,0.100000,0.100000,0.100000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000
16996,0.476190,0.047619,0.190476,0.285714,0.263158,0.052632,0.105263,0.105263,0.052632,0.000000,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000
16997,0.269841,0.190476,0.206349,0.333333,0.081967,0.065574,0.049180,0.081967,0.065574,0.000000,...,0.016949,0.050847,0.050847,0.016949,0.016949,0.016949,0.033898,0.033898,0.016949,0.000000
16998,0.255814,0.174419,0.290698,0.279070,0.071429,0.047619,0.059524,0.071429,0.059524,0.023810,...,0.000000,0.012195,0.060976,0.048780,0.024390,0.036585,0.000000,0.000000,0.048780,0.036585


In [87]:
def gc_content(dna_sequence):
    gc_percent = []
    for i in dna_sequence:
        gc_count = i.count('G') + i.count('C')
        total_count = len(i)
        gc_percent_eve = (gc_count / total_count) * 100
        gc_percent.append(gc_percent_eve)
    return gc_percent

In [88]:
# gc_precent = gc_content(data.iloc[:,0])
gc_precent = gc_content(data.seq)
gc_precent_pd = pd.DataFrame(gc_precent)
gc_precent_pd.rename(columns={0: 'GC content'}, inplace=True)
print(gc_precent_pd.shape)
gc_precent_pd

feature.insert(0,'GC content',gc_precent_pd)

feature = pd.DataFrame(feature)
feature.iloc[:,0] = feature.iloc[:,0]/100
feature

(17000, 1)


,GC content,A,C,G,T,AA,AC,AG,AT,CA,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.783525,0.105364,0.394636,0.388889,0.111111,0.011538,0.042308,0.038462,0.011538,0.025000,...,0.025097,0.001931,0.001931,0.015444,0.017375,0.001931,0.000000,0.007722,0.000000,0.000000
1,0.670732,0.152439,0.400407,0.270325,0.176829,0.026531,0.055102,0.044898,0.024490,0.071429,...,0.014344,0.012295,0.008197,0.008197,0.014344,0.014344,0.004098,0.014344,0.006148,0.024590
2,0.572727,0.236364,0.290909,0.281818,0.190909,0.050835,0.055387,0.071320,0.058422,0.068285,...,0.015957,0.013678,0.025076,0.014438,0.017477,0.009119,0.003799,0.012158,0.008359,0.012918
3,0.643377,0.187773,0.340611,0.302766,0.168850,0.021898,0.042336,0.094891,0.029197,0.075912,...,0.013177,0.013177,0.008785,0.029283,0.014641,0.004392,0.001464,0.014641,0.007321,0.000000
4,0.521886,0.262626,0.259259,0.262626,0.215488,0.047458,0.077966,0.077966,0.057627,0.101695,...,0.010239,0.000000,0.023891,0.034130,0.027304,0.006826,0.010239,0.020478,0.013652,0.010239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,0.545455,0.318182,0.090909,0.454545,0.136364,0.100000,0.050000,0.100000,0.100000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000
16996,0.238095,0.476190,0.047619,0.190476,0.285714,0.263158,0.052632,0.105263,0.105263,0.052632,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000
16997,0.396825,0.269841,0.190476,0.206349,0.333333,0.081967,0.065574,0.049180,0.081967,0.065574,...,0.016949,0.050847,0.050847,0.016949,0.016949,0.016949,0.033898,0.033898,0.016949,0.000000
16998,0.465116,0.255814,0.174419,0.290698,0.279070,0.071429,0.047619,0.059524,0.071429,0.059524,...,0.000000,0.012195,0.060976,0.048780,0.024390,0.036585,0.000000,0.000000,0.048780,0.036585


In [89]:
sequence_len = []
for i in range(data.shape[0]):
    sequence_len.append(len(data.seq[i]))
sequence_len = pd.DataFrame(sequence_len)
sequence_len.rename(columns={0:'seq_len'},inplace=True)
sequence_len

feature.insert(1,'seq_len',sequence_len)
feature.rename(columns={'GC content':'GC_con'},inplace=True)
print(feature.shape)
feature

(17000, 86)


,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.783525,522,0.105364,0.394636,0.388889,0.111111,0.011538,0.042308,0.038462,0.011538,...,0.025097,0.001931,0.001931,0.015444,0.017375,0.001931,0.000000,0.007722,0.000000,0.000000
1,0.670732,492,0.152439,0.400407,0.270325,0.176829,0.026531,0.055102,0.044898,0.024490,...,0.014344,0.012295,0.008197,0.008197,0.014344,0.014344,0.004098,0.014344,0.006148,0.024590
2,0.572727,1320,0.236364,0.290909,0.281818,0.190909,0.050835,0.055387,0.071320,0.058422,...,0.015957,0.013678,0.025076,0.014438,0.017477,0.009119,0.003799,0.012158,0.008359,0.012918
3,0.643377,687,0.187773,0.340611,0.302766,0.168850,0.021898,0.042336,0.094891,0.029197,...,0.013177,0.013177,0.008785,0.029283,0.014641,0.004392,0.001464,0.014641,0.007321,0.000000
4,0.521886,297,0.262626,0.259259,0.262626,0.215488,0.047458,0.077966,0.077966,0.057627,...,0.010239,0.000000,0.023891,0.034130,0.027304,0.006826,0.010239,0.020478,0.013652,0.010239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,0.545455,22,0.318182,0.090909,0.454545,0.136364,0.100000,0.050000,0.100000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000
16996,0.238095,21,0.476190,0.047619,0.190476,0.285714,0.263158,0.052632,0.105263,0.105263,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000
16997,0.396825,63,0.269841,0.190476,0.206349,0.333333,0.081967,0.065574,0.049180,0.081967,...,0.016949,0.050847,0.050847,0.016949,0.016949,0.016949,0.033898,0.033898,0.016949,0.000000
16998,0.465116,86,0.255814,0.174419,0.290698,0.279070,0.071429,0.047619,0.059524,0.071429,...,0.000000,0.012195,0.060976,0.048780,0.024390,0.036585,0.000000,0.000000,0.048780,0.036585


# cds特征

In [90]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\c_cds_test\Sorghum_bicolor',names=names,sep='\t', index_col=index_col)
rna_cds = rna_cds
rna_cds_nc = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\\nc_cds_test\Sorghum_bicolor',names=names,sep='\t', index_col=index_col)
rna_cds_nc = rna_cds_nc

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)
rna_cds

data1 = data
data2 = data
for i, l in data2.iterrows():
    data1.iloc[i,2] = len(l['seq'])
    
mylist = []
mylist = pd.DataFrame(mylist,columns=['score','cdsStarts','cdsStop','cdsSizes','cdsPercent'])

for index, row in rna_cds.iterrows():
    tem_pd_score = pd.DataFrame(row)
    mylist.loc[index, 'cdsStarts'] = row["start"]
    mylist.loc[index, 'cdsStop'] = row["end"]
    mylist.loc[index, 'cdsSizes'] = row["end"] - row["start"]
    mylist.loc[index, 'score'] = tem_pd_score.iloc[4, 0]
    mylist.loc[index, 'cdsPercent'] = (row["start"] + row["end"]) / data1.loc[index, 'seq']
    
feature = feature.join(mylist)

# 选取其中10个特征
# feature = feature[['GC_con', 'score', 'cdsStop', 'cdsSizes','T','C','GT','GC','ACG','TAT']]
# 选取其中45个特征
# feature = feature[['GC_con', 'seq_len', 'A', 'C', 'G', 'T', 'AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT', 'AAA', 'AAG', 'AGA', 'ATT', 'CCG', 'CGC', 'CGG', 'CTC', 'GAA', 'GAG', 'GCC', 'GCG', 'GGC', 'GGG', 'TCT', 'TGG', 'TTG', 'TTT', 'score', 'cdsStarts', 'cdsStop', 'cdsSizes', 'cdsPercent']]

feature

,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,TAT
0,0.783525,722.0,522,522,0.111111,0.394636,0.046154,0.151923,0.019305,0.000000
1,0.670732,592.0,492,492,0.176829,0.400407,0.051020,0.104082,0.016393,0.004098
2,0.572727,1520.0,1320,1320,0.190909,0.290909,0.031108,0.093323,0.008359,0.009119
3,0.643377,787.0,687,687,0.168850,0.340611,0.042336,0.118248,0.016105,0.005857
4,0.521886,452.0,297,252,0.215488,0.259259,0.037288,0.088136,0.010239,0.000000
...,...,...,...,...,...,...,...,...,...,...
16995,0.545455,198.0,17,6,0.136364,0.090909,0.050000,0.050000,0.000000,0.000000
16996,0.238095,165.0,20,15,0.285714,0.047619,0.105263,0.000000,0.000000,0.058824
16997,0.396825,74.0,61,24,0.333333,0.190476,0.081967,0.032787,0.000000,0.016949
16998,0.465116,243.5,51,45,0.279070,0.174419,0.047619,0.083333,0.012195,0.000000


## 深度特征

In [91]:
# data=pd.read_csv(r'C:\Users\26970\Desktop\特征融合\基准数据集.csv')
# sequences = data['Sequence']
# labels = data.Label.values

sequences = data3['seq']
labels = data3['label'].values

print(labels)

pat = re.compile('[AGCTagct]')

def pre_process(text):
    text = pat.findall(text)
    text = [each.lower() for each in text]
    return text

x = sequences.apply(pre_process)
#print(x)
word_set = set()

for lst in x:
    for word in lst:
        word_set.add(word)

word_list = list(word_set)
word_index = dict([(each, word_list.index(each) + 1) for each in word_list])
#print(word_list)
#print(word_index)

text = x.apply(lambda x: [word_index.get(word, 0) for word in x])
print(text)

text_len = 1200

pad_text = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text]
pad_text = np.array(pad_text)
pad_text

pad_text,labels=torch.LongTensor(pad_text),torch.LongTensor(labels)
# x_train, x_test, y_train, y_test = train_test_split(pad_text, labels, test_size=0.3)


#x_train=x_train.reshape(32,1000,-1)

#y_train=to_categorical(y_train,2)
#y_test=to_categorical(y_test,2)
#pad_text,labels=torch.tensor(pad_text).float(),torch.LongTensor(labels)

#构建数据迭代器
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, text_list, label_list):  # 将序列和标签保存在类属性中
        self.text_list = text_list
        self.label_list = label_list

    def __getitem__(self, index):  # 索引序列和标签
        text = torch.LongTensor(self.text_list[index])  # Tensor中的元素转换为64位整型
        label = self.label_list[index]
        return text, label

    def __len__(self):
        return len(self.text_list)  # 返回序列数

data_ds = Mydataset(pad_text,labels)
#train_ds = Mydataset(x_train, y_train)  # 将训练集单独存储
#test_ds = Mydataset(x_test, y_test)  # 将测试集单独存储

batch_size = 64  # 批量大小
print(len(data_ds))
data_dl =torch.utils.data.DataLoader(data_ds,batch_size=batch_size,shuffle=False)
#train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
#test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

#定义网络架构
embed_dim = 24  # 嵌入维度
hidden_size = 20  # 隐藏层单元数


class Net(nn.Module):
    def __init__(self, word_list, embed_dim, hidden_size):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(word_list) + 1, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_size)
        self.pool1=nn.MaxPool1d(5)
        self.pool2=nn.MaxPool1d(3)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, inputs):
        bz = inputs.shape[1]
        h0 = torch.zeros((1, bz, hidden_size)).to(device)
        c0 = torch.zeros((1, bz, hidden_size)).to(device)
        x = self.em(inputs)                                         #[64,1200,24]
        r_o, _ = self.rnn(x, (h0, c0))                              #[64,1200,24]
        x = self.pool1(x)                                            #[64,1200,4]
        x = self.pool2(x)                                            #[64,1200,1]
        x = x.view(-1,1200) 
        #r_o = r_o[-1]                                               #[64,1200,24]
#         x = x.view(64,-1)                                           #[64,28800]  两层池化层之后---->[64,1200]
        #x = F.dropout(F.relu(self.fc1(r_o)))                       #[1200,128]
        #x = self.fc2(x)

        return x

#定义交叉熵损失函数和Adam优化器
model = Net(word_list, embed_dim, hidden_size)  # 实例化
model = model.to(device)

loss = nn.CrossEntropyLoss()  # 默认求每个batch下的平均损失
loss = loss.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

with torch.no_grad():
    outputs=[]
    for x,y in data_dl:
        out = model(x)
        outputs.append(out)
#     print(outputs)
#     print(outputs[0].shape)
#     print(len(outputs))
#r1 = [i.reshape(16000,1) for i in out]
# print(outputs)
outputs1 = torch.concat(outputs,axis =0)
# print(outputs1)

deep_fea=pd.DataFrame(outputs1)

deep_fea.to_csv(r'C:\Users\26970\Desktop\特征融合\10特征\Sorghum_bicolor-lstm.csv')

[0 0 0 ... 1 1 1]
0        [3, 4, 2, 2, 3, 1, 1, 2, 1, 4, 4, 1, 1, 4, 2, ...
1        [3, 4, 2, 4, 2, 2, 3, 1, 1, 1, 3, 2, 3, 2, 2, ...
2        [3, 4, 2, 2, 1, 2, 4, 2, 2, 1, 4, 1, 2, 1, 2, ...
3        [3, 4, 2, 4, 1, 1, 2, 4, 1, 1, 4, 4, 1, 4, 1, ...
4        [3, 1, 1, 2, 3, 2, 4, 4, 1, 1, 4, 2, 2, 2, 1, ...
                               ...                        
16995    [2, 2, 1, 3, 4, 2, 2, 2, 3, 3, 1, 3, 4, 2, 4, ...
16996    [3, 3, 2, 4, 4, 3, 4, 2, 3, 3, 1, 3, 4, 3, 3, ...
16997    [2, 4, 1, 4, 4, 3, 1, 3, 1, 4, 4, 2, 3, 4, 1, ...
16998    [4, 2, 2, 3, 4, 2, 3, 4, 2, 3, 3, 3, 4, 2, 1, ...
16999    [4, 4, 3, 3, 3, 1, 2, 3, 3, 1, 3, 3, 3, 1, 4, ...
Name: seq, Length: 17000, dtype: object
17000


In [92]:
fea_all = feature.join(deep_fea)
fea_all

,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,TAT,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.783525,722.0,522,522,0.111111,0.394636,0.046154,0.151923,0.019305,0.000000,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
1,0.670732,592.0,492,492,0.176829,0.400407,0.051020,0.104082,0.016393,0.004098,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
2,0.572727,1520.0,1320,1320,0.190909,0.290909,0.031108,0.093323,0.008359,0.009119,...,1.359442,2.290176,1.753113,1.359442,1.696464,2.290176,2.290176,1.696464,2.290176,1.753113
3,0.643377,787.0,687,687,0.168850,0.340611,0.042336,0.118248,0.016105,0.005857,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
4,0.521886,452.0,297,252,0.215488,0.259259,0.037288,0.088136,0.010239,0.000000,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,0.545455,198.0,17,6,0.136364,0.090909,0.050000,0.050000,0.000000,0.000000,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
16996,0.238095,165.0,20,15,0.285714,0.047619,0.105263,0.000000,0.000000,0.058824,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
16997,0.396825,74.0,61,24,0.333333,0.190476,0.081967,0.032787,0.000000,0.016949,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
16998,0.465116,243.5,51,45,0.279070,0.174419,0.047619,0.083333,0.012195,0.000000,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620


In [93]:
y = pd.DataFrame(label_data, columns=['label'])

fea_all.insert(0,'label',y)
fea_all

,label,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0,0.783525,722.0,522,522,0.111111,0.394636,0.046154,0.151923,0.019305,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
1,0,0.670732,592.0,492,492,0.176829,0.400407,0.051020,0.104082,0.016393,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
2,0,0.572727,1520.0,1320,1320,0.190909,0.290909,0.031108,0.093323,0.008359,...,1.359442,2.290176,1.753113,1.359442,1.696464,2.290176,2.290176,1.696464,2.290176,1.753113
3,0,0.643377,787.0,687,687,0.168850,0.340611,0.042336,0.118248,0.016105,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
4,0,0.521886,452.0,297,252,0.215488,0.259259,0.037288,0.088136,0.010239,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,1,0.545455,198.0,17,6,0.136364,0.090909,0.050000,0.050000,0.000000,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
16996,1,0.238095,165.0,20,15,0.285714,0.047619,0.105263,0.000000,0.000000,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
16997,1,0.396825,74.0,61,24,0.333333,0.190476,0.081967,0.032787,0.000000,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620
16998,1,0.465116,243.5,51,45,0.279070,0.174419,0.047619,0.083333,0.012195,...,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620,1.913620


In [17]:
# 定义朴素贝叶斯模型
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict

def model_gnb(X, y, X_test, y_test):
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    gnb = GaussianNB()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行五倍交叉验证
    cv_results = cross_validate(gnb, X, y, scoring=scoring, cv=5, return_train_score=False)

    # 计算并输出评估指标
    # print("Accuracy: %f (+/- %f)" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std() * 2))
    # print("F1 score: %f (+/- %f)" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std() * 2))
    # print("ROC AUC score: %f (+/- %f)" % (cv_results['test_roc_auc'].mean(), cv_results['test_roc_auc'].std() * 2))
    # print("MCC: %f (+/- %f)" % (cv_results['test_mcc'].mean(), cv_results['test_mcc'].std() * 2))

    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(gnb, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")
    
        # 在独立测试集上评估模型
    gnb.fit(X, y)
    y_pred_test = gnb.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")

model_gnb(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Accuracy: 0.766125
F1 score: 0.791149
ROC AUC score: 0.777379
MCC: 0.553771


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

NPV: 0.854715
PPV: 0.712943
Sensitivity (Recall): 0.891000
Specificity: 0.641250


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Set - Accuracy: 0.537060
Test Set - F1 score: 0.000000
Test Set - ROC AUC score: 0.500000
Test Set - MCC: 0.000000
Test Set - NPV: 0.537060
Test Set - PPV: nan
Test Set - Sensitivity (Recall): 0.000000
Test Set - Specificity: 1.000000


C:\Users\26970\AppData\Local\Temp\ipykernel_46192\1892360647.py:67: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv_test = tp_test / (tp_test + fp_test)


In [18]:
# 定义SVM模型

def model_SVM(X, y, X_test, y_test):
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建SVM分类器
    svm_clf = SVC()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(svm_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(svm_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    svm_clf.fit(X, y)
    y_pred_test = svm_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_SVM(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [94]:
### RFC训练模型

def model_RFC(X, y, X_test, y_test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict

    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    # 创建随机森林分类器
    rf_clf = RandomForestClassifier()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(rf_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(rf_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")
    
    # 在独立测试集上评估模型
    rf_clf.fit(X, y)
    y_pred_test = rf_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")

model_RFC(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarni

Accuracy: 0.939000
F1 score: 0.942137
ROC AUC score: 0.955783
MCC: 0.883131


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionW

NPV: 0.991459
PPV: 0.896252
Sensitivity (Recall): 0.992375
Specificity: 0.885125


C:\Users\26970\AppData\Local\Temp\ipykernel_24320\935470792.py:45: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X, y)


Test Set - Accuracy: 0.851824
Test Set - F1 score: 0.864431
Test Set - ROC AUC score: 0.853803
Test Set - MCC: 0.722723
Test Set - NPV: 0.953578
Test Set - PPV: 0.784584
Test Set - Sensitivity (Recall): 0.962373
Test Set - Specificity: 0.745234
